In [ ]:
import pandas as pd
import requests
import sys
def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()
    if len(json["resultSets"])== 1:

        
        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        print(data)
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)
    return df
rs_frames=[]


for year in range(1997,2025):
    season = f"{year-1}-{str(year)[-2:]}"
    stype = 'Regular%20Season'
    url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Four%20Factors&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df = pull_data(url)
    url2='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df2 = pull_data(url2)
    df=df.merge(df2)
    url3 ='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Opponent&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df3 = pull_data(url3)
    df3col=['TEAM_ID', 'OPP_FTA','OPP_FGA',  'OPP_PTS']
    df3=df3[df3col]
    df3['OPP_TS_PCT'] = df3['OPP_PTS'] / (2 * (df3['OPP_FGA'] + (0.44 * df3['OPP_FTA'])))

    df=df.merge(df3,on='TEAM_ID')
    #print(df)
    #sys.exit()
    df['year']=year
    rs_frames.append(df)
ps_frames=[]
for year in range(1997,2025):
    season = f"{year-1}-{str(year)[-2:]}"
    stype = 'Playoffs'
    url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Four%20Factors&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df = pull_data(url)
    url2='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
    df2 = pull_data(url2)
    df=df.merge(df2)
    url3 ='https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Opponent&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType='+stype+'&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

    df3 = pull_data(url3)
    df3col=['TEAM_ID', 'OPP_FTA',  'OPP_FGA',  'OPP_PTS']
    df3=df3[df3col]
    df=df.merge(df3,on='TEAM_ID')
    df['year']=year
    ps_frames.append(df)


In [ ]:
rs_ff=pd.concat(rs_frames)
ps_ff=pd.concat(ps_frames)

ps_ff[ps_ff.year==2020]

In [ ]:
rs_ff=pd.concat(rs_frames)
rs_ff.to_csv('teamfactors.csv',index = False)

ps_ff=pd.concat(ps_frames)
ps_ff.to_csv('teamfactors_ps.csv',index = False)

In [ ]:
import pandas as pd
rs_ff = pd.read_csv('teamfactors.csv')
print(rs_ff.columns)
ps_ff = pd.read_csv('teamfactors_ps.csv')

def sum_factors(rs_ff):
    frames=[]
    for year in range(1997,2025):
        year_ff=rs_ff[rs_ff.year==year].reset_index(drop=True)
        
        year_ff['OPP_TS_PCT'] = year_ff['OPP_PTS'] / (2 * (year_ff['OPP_FGA'] + (0.44 * year_ff['OPP_FTA'])))

        columns = ['TEAM_ID', 'TEAM_NAME', 'TS_PCT','EFG_PCT',
           'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT', 'OPP_TS_PCT','OPP_EFG_PCT', 'OPP_FTA_RATE',
           'OPP_TOV_PCT', 'OPP_OREB_PCT','NET_RATING','OFF_RATING', 'DEF_RATING', 'year']
        year_ff=year_ff[columns]
        avg_rating=year_ff['OFF_RATING'].sum()/len(year_ff)
    
        year_ff['RDEF_RATING'] = year_ff['DEF_RATING']- avg_rating
        year_ff['ROFF_RATING'] = year_ff['OFF_RATING'] -  avg_rating
        frames.append(year_ff)
    total = pd.concat(frames)
    return total

rs_df = sum_factors(rs_ff)
rs_df.to_csv('fourfactors.csv')
ps_df = sum_factors(ps_ff)
ps_df.to_csv('fourfactors_ps.csv')
print(rs_df.columns)

In [ ]:
for year in range(1997,2025):
    print(year)
    
    yrs_df = rs_df[rs_df.year==year]
    #print('Year Min TS')
    #print(yrs_df['TS_PCT'].min())
    print('Year Min OPP TS')
    print(yrs_df['OPP_TS_PCT'].min())
    #print('Year Max TS')
    #print(yrs_df['TS_PCT'].max())
    print('Year max OPP TS')
    print(yrs_df['OPP_TS_PCT'].max())